***Introduction***

This document provides a detailed overview of the methods and calculations essential for converting raw sensor currents into accurate gas concentration readings using Alphasense sensors. As leaders in gas detection technology, Alphasense provides robust solutions that are crucial for a variety of applications ranging from environmental monitoring to industrial safety. The focus here is to elucidate the specific equations and algorithms that are fundamental to the sensor data interpretation process, enabling users to transform electrical signals emitted by the sensors into meaningful data.

Contained within, you will find a comprehensive exploration of the mathematical models and conversion formulas as provided by Alphasense, designed to facilitate the accurate calculation of gas concentrations from the raw currents measured by the sensors. This documentation is intended to be a valuable resource for engineers, researchers, and professionals who rely on precise and dependable gas measurement techniques. Through an in-depth understanding of these conversion principles, users can ensure the effective application of Alphasense technology in their respective fields, thereby enhancing the reliability and accuracy of their gas detection systems.

-------------------------------------------------------------
***Terms definitions***

- WEu = uncorrected raw WE output (mV - coming from BU)
- AEu = uncorrected raw AE output (mV - coming from BU)
- WEc = corrected WE output (Result Output)
- WEe = WE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEe = AE electronic offset on the AFE or ISB (mV - initially provided by Alphasense - source from AvSystem on runtime)
- WEo = WE sensor zero, i.e. the sensor WE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)
- AEo = AE sensor zero, i.e. the sensor AE output in zero air (mV - initially provided by Alphasense - source from AvSystem on runtime)

- WET = Total WE zero offset
- AET = Total AE zero offset

- nT = temperature dependent correction factor for algorithm 1, refer to Table 3 for values
- kT = temperature dependent correction factor for algorithm 2, refer to Table 3 for values
- k'T = temperature dependent correction factor for algorithm 3, refer to Table 3 for values
- k''T = temperature dependent correction factor for algorithm 4, refer to Table 3 for values

---------------------------------------------------------

***Algorithms***

- Algorithm 1
    ```
    WEc = (WEu - WEe) - nT * (AEu - AEe)
    ```
- Algorithm 2
    ```
    WEc = (WEu - WEe) - kT * (WEo / AEo) * (AEu - AEe)
    ```
- Algorithm 3
    ```
    WEc = (WEu - WEe) - (WEo - AEo) - k'T * (AEu - AEe)
    ```
- Algorithm 4
    ```
    WEc = (WEu - WEe) - WEo - k''T
    ```

-----------------------------------------------------------
***Sensorbee Sensors***

- CO-B4
    - Type: B
    - Recommended Algorithm: Algorithm 1

- H2S-B4
    - Type: B
    - Recommended Algorithm: 1

- NO-B4
    - Type: B
    - Recommended Algorithm: 2

- NO2-B43F
    - Type: B
    - Recommended Algorithm: 1

- OX-B431
    - Type: B
    - Recommended Algorithm: 1

- SO2-B4
    - Type: B
    - Recommended Algorithm: 4


----------------------------------------------------
***JavaScript Source Code Library***

In [19]:
var converter = {};
var globalParam = {};

(function(context) {
    // list of available algorithms from alphasense
    context.algorithms = {
        1: {
            parameters: ['WEu', 'WEe', 'AEu', 'AEe']
        },
        2: {
            parameters: ['WEu', 'WEe', 'WEo', 'AEo', 'AEu', 'AEe']
        },
        3: {
            parameters: ['WEu', 'WEe', 'WEo', 'AEo', 'AEu', 'AEe']
        },
        4: {
            parameters: ['WEu', 'WEe', 'WEo']
        }
    
    }
    
    // gas types definition
    context.gasTypes = {
        'CO-B4': {
            algo: 1,
            coefficients: [0.7, 0.7, 0.7, 0.7, 1, 3, 3.5, 4, 4.5]
        }, 
        'H2S-B4': {
            algo: 1,
            coefficients: [-0.6, -0.6, 0.1, 0.8, -0.7, -2.5, -2.5, -2.2, -1.8]
        }, 
        'NO-B4': {
            algo: 2,
            coefficients: [2.9, 2.9, 2.2, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3]
        },
        'NO2-B43F': {
            algo: 1,
            coefficients: [1.3, 1.3, 1.3, 1.3, 1, 0.6, 0.4, 0.2, -1.5]
           
        },
        'OX-B431': {
            algo: 1,
            coefficients: [0.9, 0.9, 1, 1.3, 1.5, 1.7, 2, 2.5, 3.7]
        },
        'SO2-B4': {
            algo: 4,
            coefficients: [-4, -4, -4, -4, -4, 0, 20, 140, 450]
        }
    };
    
    // evaluate polynomial eq, where x = temperature
    context.evaluatePolynomial  = function (coeffs, x) {
        if (coeffs.length !== 9) {
            throw new Error("Invalid number of coefficients. Exactly 9 coefficients are required.");
        }
    
        let result = 0;
        coeffs.forEach((coeff, index) => {
            const power = coeffs.length - 1 - index;  
            result += coeff * Math.pow(x, power);  
        });

        return result / 100;
    };
    
    // validate if parameters needed are complete
    context.validateParameters = function(checkList, parameters) {
        const total = checkList.length;
        let ctr = 0;
        
        checkList.forEach((value, index) => {
            if (parameters[value]) {
                ctr++;
            }
        });
    
        if (ctr === total) {
            return true;
        } else {
            return false;
        }
    };
    
    // alphaSense algorithm 1
    context.algo1 = function (parameters, nT, WEc_no2 = 0) {
        let result = 0;
        const { WEu, WEe, AEu, AEe } = parameters;
        result = (WEu - (WEe + WEc_no2)) - nT * (AEu - AEe);
        return result;
    };

    // alphaSense algorithm 2
    context.algo2 = function (parameters, kT) {
        let result = 0;
        const { WEu, WEe, WEo, AEo, AEu, AEe } = parameters;
        result = (WEu - WEe) - kT * (WEo / AEo) * (AEu - AEe);
        return result;
    };

    // alphasense algorithm 3
    context.algo3 = function (parameters, kT, WEc_no2 = 0) {
        let result = 0;
        const { WEu, WEe, WEo, AEo, AEu, AEe } = parameters;
        result = (WEu - (WEe + WEc_no2)) - (WEo - AEo) - kT * (AEu - AEe);
        return result;
    };
    
    // alphaSense algorithm 4
    context.algo4 = function (parameters, kT) {
        let result = 0;
        const { WEu, WEe, WEo } = parameters;
        result = (WEu - WEe) - WEo - kT;
        return result;
    };

    // convert corrected voltage to ppm
    context.convertToPpb = function(corrected, sensitivity) {
        const ppb = corrected / sensitivity;
        return ppb;
    };
    
    // convert to gas
    context.convertToTargetGas = function (type, temperature, we_sensor, gain, parameters, no2_ppb = null, no2_sensitivity = null) {
        const { algo, coefficients } = context.gasTypes[type];
        const sensitivity = we_sensor * gain;
        const calculations = {
            1: context.algo1,
            2: context.algo2,
            3: context.algo3,
            4: context.algo4
        };
        const validate = context.validateParameters(context.algorithms[algo].parameters, parameters);
        
        if (type !== 'OX-B431') {
            // non- O3
            if (validate) {
                const factor = context.evaluatePolynomial(coefficients, temperature);
                const WEc = calculations[algo](parameters, factor);
                const ppb = context.convertToPpb(WEc, sensitivity);
                
                return {
                    WEc,
                    correctionFactor: factor,
                    ppb
                };
            } else {
                 throw new Error("Incomplete list of parameters.");
            }
        } else {
            // To extract O3 from OX-B431, the process requires no2 ppb, and no2 sensitivity
            // no2 ppb can be obtained using this function - just pass the necessary parameters for type NO2-B43F
            // no2 sensitivity is provided by alphasense
            if (validate && no2_ppb !== null && no2_sensitivity !== null && (algo === 1 || algo === 3)) {
                const factor = context.evaluatePolynomial(coefficients, temperature);
                const WEc_no2 = no2_ppb * no2_sensitivity;
                const WEc = calculations[algo](parameters, factor, WEc_no2);
                const ppb = context.convertToPpb(WEc, sensitivity);

                return {
                    WEc,
                    correctionFactor: factor,
                    ppb
                };
            } else {
                if (!validate) {
                    throw new Error("Incomplete list of parameters.");
                } else if (no2_ppb === null) {
                    throw new Error("Missing No2 concentration.");
                } else if (no2_sensitivity === null) {
                    throw new Error("Missing No2 sensitivity.");
                } else if (algo !== 1 && algo !== 3) {
                    throw new Error("Invalid algorithm for O3.");
                }
            }
        }
    };
})(converter);


--------------------------------------

***Expected Parameters / Sources:***

***Electrodes***:
- ***WEe*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***AEe*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***WEo*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***AEo*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***WEu*** - to be sourced from AvSystem as raw voltage during runtime
- ***AEu*** - to be sourced from AvSystem as raw voltage during runtime

***Values / Settings*** 
- ***Temperature*** - to be sourced from AvSystem temperature object
- ***we_sensor (Sensitivity (nA/ppm))*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime
- ***gain*** - initially from AlphaSense Calibration file, but will be sourced from AvSystem during runtime


---------------------------------------------------------------------------------------


***Test Environment Arguments/Parameters***

Enter Your Arguments/Parameters below (Update the values to see changes in the result):

In [22]:
globalParam.WEu = 1.6;
globalParam.WEe = 233;
globalParam.AEu = 1.4;
globalParam.AEe = 243;
globalParam.AEo = 241;
globalParam.WEo = 234;

globalParam.sensor_type = 'OX-B431'; // 'NO2-B43F';
globalParam.temperature = 2.5;
globalParam.we_sensor = -494;
globalParam.gain = -0.73;

-0.73


***Test Environment***

In [23]:
var result = converter.convertToTargetGas(globalParam.sensor_type, globalParam.temperature, globalParam.we_sensor, globalParam.gain, {
    'WEu': globalParam.WEu, 
    'WEe': globalParam.WEe, 
    'AEu': globalParam.AEu, 
    'AEe': globalParam.AEe,
    'AEo': globalParam.AEo,
    'WEo': globalParam.WEo
}, 1.0668287930339655, -580);

console.log('result', result);

result {
  WEc: 7672.237281250001,
  correctionFactor: 32.71373046875,
  ppb: 21.275129724502246
}
